# Latihan Hyperparameter Tuning Klasifikasi

In [1]:
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Mengunduh dataset German Credit dari OpenML
X, y = fetch_openml(name='credit-g', version=1, return_X_y=True, as_frame=True)

# Konversi target menjadi numerik 
le = LabelEncoder()
y = le.fit_transform(y) # Mengubah 'good' menjadi 1 dan 'bad' menjadi 0

# Melakukan One-Hot Encoding pada fitur kategorikal 
X_encoded = pd.get_dummies(X, drop_first=True) # konversi fitur kategorikal menjadi numerik

# Membagi dataset menjadi training set dan testing set (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

print("Shape of training data:", X_train.shape)
print("Shape of testing data:", X_test.shape)

Shape of training data: (700, 48)
Shape of testing data: (300, 48)


In [2]:
from sklearn.ensemble import RandomForestClassifier

# Inisialisasi model Random Forest tanpa hyperparameter tuning
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluasi awal model tanpa tuning
initial_score = rf.score(X_test, y_test)
print(f'Initial laccuracy on test set (without tuning): {initial_score:.2f}')

Initial laccuracy on test set (without tuning): 0.76


## Grid Search 


In [3]:
from sklearn.model_selection import GridSearchCV

# Definisikan parameter grid untuk Grid Search 
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Output hasil terbaik 
print(f'Best Parameters (Grid Search): {grid_search.best_params_}')
best_rf_grid = grid_search.best_estimator_

# Evaluasi peforma model saat test set 
grid_search_score = best_rf_grid.score(X_test, y_test)
print(f"Accuracy after grid search: {grid_search_score:.2f}")

Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best Parameters (Grid Search): {'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 5, 'n_estimators': 200}
Accuracy after grid search: 0.76


## Random Search

In [4]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Definisinak ruang pencarian untuk random search
param_dist = {
    'n_estimators': np.linspace(100, 500, 5, dtype=int),
    'max_depth': np.linspace(10, 50, 5, dtype=int),
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Inisialisasi RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=20, cv=3, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

# Output hasil terbaik 
print(f"Best parameters (Random Search): {random_search.best_params_}")
best_rf_random = random_search.best_estimator_

# Evaluasi peforma model pada saat test set
random_search_score = best_rf_random.score(X_test, y_test)
print(f'Evaluasi After random search: {random_search_score:.2f}')

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best parameters (Random Search): {'n_estimators': np.int64(200), 'min_samples_split': 5, 'max_depth': np.int64(30), 'criterion': 'gini'}
Evaluasi After random search: 0.76


## Bayesian Optimization

In [5]:
from skopt import BayesSearchCV

# Definisikan ruang pencarian untuk Bayesian Optimization
param_space = {
    'n_estimators': (100, 500),
    'max_depth': (10, 50),
    'min_samples_split': (2, 10),
    'criterion': ['gini', 'entropy']
}

# Inisialisasi BayesSearchCV
bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=32, cv=3, n_jobs=-1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)

# Output hasil terbaik
print(f"Best parameters (Bayesian Optimization): {bayes_search.best_params_}")
best_rf_bayes = bayes_search.best_estimator_
 
# Evaluasi performa model pada test set
bayes_search_score = best_rf_bayes.score(X_test, y_test)
print(f"Accuracy after Bayesian Optimization: {bayes_search_score:.2f}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

seperti yang sudah kita bahas penurunan akurasi setelah hyperparameter tuning adalah hal yang umum terjadi dalam pembangunan model machine learning, dan beberapa faktor yang berkontribusi terhadap maslah ini anatara lain sebagai berikut

1. Overfitting: tuning hyperparameter secara berlebihan bisa menyebabkakn model selalu sesuai dengan data pelatihan (overfit). hal ini menyebabkan peforma model yang baik pada data pelatihan tetapi buruk pada data pengujian
2. Underfitting: sebaliknya, jika model tidak cukup kompleks setelah tuning, model bisa gagal menangkap pola penting dari data (underfit).
3. Pemilihan Ruang Pencarian yang Tidak Tepat: jika rentang atau ruang hyperparameter yang dicari terlalu kecil atau tidak mencakup area yang optimal, hasil tuning dapat merugikan performa model.
4. Evaluasi yang Tidak Konsisten: cross-validation yang kurang efektif atau ruang pencarian hyperparameter yang terlalu besar juga dapat membuat tuning kurang efektif.

untuk melakukan tuning yang lebih baik, anda dapat meerapakan beberapa saran berikut
1. memastikan bahwa dataset yang digunakan sudah baik
2. memperluas ruang pencarian hyperparamter
3. memfokuskan tuning pada hyperparametr kunci yang aling berdampak pada peforma model, seperti n_estimators, max_depth, dan min_samples_split.
4. Menggunakan cross-validation dengan lebih banyak fold untuk membuat evaluasi performa lebih stabil.
5. Memperhatikan keseimbangan antara eksplorasi dan eksploitasi hyperparameter dengan menggunakan metode seperti Bayesian Optimization yang lebih efisien dalam memprediksi kombinasi optimal.